In [1]:
import numpy as np
mean = np.array([1,2,3])
# define covariance matrix of size 3x3, with random diagonal values in the range [0,1]
cov = np.diag(np.random.rand(3))

controls = np.array([1,2])
measurements = np.array([1,2,3])
mean, cov, controls, measurements

(array([1, 2, 3]),
 array([[0.19279048, 0.        , 0.        ],
        [0.        , 0.79335283, 0.        ],
        [0.        , 0.        , 0.74230311]]),
 array([1, 2]),
 array([1, 2, 3]))

In [7]:
delta_t= 1

# prediction step
# state transition matrix A (nxn): n is the number of state variables
A = np.identity(len(mean))
# get orinetation of the robot
theta = mean[2]
# Control matrix B (nxm), where m is the number of control inputs
B = np.array([[delta_t * np.cos(theta), 0],
            [delta_t * np.sin(theta), 0],
            [0, delta_t]]
            )
# measurement noise (nxn), initialize with random values
R = np.diag(np.random.rand(len(mean)))
# update the mean
mean = np.matmul(A, mean) + np.matmul(B, controls)
# update covariance matrix
cov = np.matmul(np.matmul(A, cov), np.transpose(A)) + R

A, B, R, mean, cov

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[0.75390225, 0.        ],
        [0.6569866 , 0.        ],
        [0.        , 1.        ]]),
 array([[0.77259176, 0.        , 0.        ],
        [0.        , 0.64009725, 0.        ],
        [0.        , 0.        , 0.44770526]]),
 array([1.04757194, 1.83918233, 9.        ]),
 array([[1.9892287 , 0.24220743, 0.24220743],
        [0.46555392, 1.13056753, 0.46555392],
        [0.31088046, 0.31088046, 1.41826568]]))

In [8]:
# Correction step
# measurement matrix C (kxn), where k is the number of measurements
C = np.identity(len(mean))
# measurement noise (kxk), initialize with random values
Q = np.diag(np.random.rand(len(mean)))

# Kalman gain
K = np.matmul(np.matmul(cov, np.transpose(C)), np.linalg.inv(np.matmul(np.matmul(C, cov), np.transpose(C)) + Q))
# update the mean
mean = mean + np.matmul(K, measurements - np.matmul(C, mean))
# update covariance matrix
cov = np.matmul(np.identity(len(mean)) - np.matmul(K, C), cov)

C, Q, K, mean, cov

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[0.48645595, 0.        , 0.        ],
        [0.        , 0.14649253, 0.        ],
        [0.        , 0.        , 0.58424812]]),
 array([[0.79485629, 0.03483909, 0.01671292],
        [0.02016607, 0.874983  , 0.02662543],
        [0.02576393, 0.07090908, 0.68864118]]),
 array([0.91508429, 1.81918316, 4.87833072]),
 array([[0.38666257, 0.00510367, 0.00976449],
        [0.00980991, 0.12817847, 0.01555586],
        [0.01253302, 0.01038765, 0.40233731]]))

In [13]:
def kalaman_filter(mean, cov, controls, measurements, delta_t):
    """
    Implements the Kalman filter algorithm for state estimation.

    Args:
        mean (numpy.ndarray): The mean of the state estimate.
        cov (numpy.ndarray): The covariance matrix of the state estimate.
        controls (numpy.ndarray): The control inputs applied to the system.
        measurements (numpy.ndarray): The measurements obtained from the system.
        delta_t (float): The time step between control inputs.

    Returns:
        numpy.ndarray: The updated mean of the state estimate.
        numpy.ndarray: The updated covariance matrix of the state estimate.
    """
    
    # state transition matrix A (nxn): n is the number of state variables
    A = np.identity(len(mean))
    # get orientation of the robot
    theta = mean[2]
    # Control matrix B (nxm), where m is the number of control inputs
    B = np.array([[delta_t * np.cos(theta), 0],
                [delta_t * np.sin(theta), 0],
                [0, delta_t]]
                )
    # measurement noise (nxn), initialize with random values
    R = np.diag(np.random.rand(len(mean)))
    # measurement matrix C (kxn), where k is the number of measurements
    C = np.identity(len(mean))
    # measurement noise (kxk), initialize with random values
    Q = np.diag(np.random.rand(len(mean)))
    
    # prediction step
    # update the mean
    mean = np.matmul(A, mean) + np.matmul(B, controls)
    # update covariance matrix
    cov = np.matmul(np.matmul(A, cov), np.transpose(A)) + R
    # correction step
    # Kalman gain
    K = np.matmul(np.matmul(cov, np.transpose(C)), np.linalg.inv(np.matmul(np.matmul(C, cov), np.transpose(C)) + Q))
    # update the mean
    mean = mean + np.matmul(K, measurements - np.matmul(C, mean))
    # update covariance matrix
    cov = np.matmul(np.identity(len(mean)) - np.matmul(K, C), cov)

    return mean, cov


In [20]:
mean = np.array([1,2,3])
# define covariance matrix of size 3x3, with random diagonal values in the range [0,1]
cov = np.diag(np.random.rand(3))
controls = np.array([1,2])
measurements = np.array([1,2,3])

kalaman_filter(mean, cov, controls, measurements, delta_t)

(array([0.94855665, 2.05930592, 3.62918767]),
 array([[0.04314669, 0.        , 0.        ],
        [0.        , 0.2562228 , 0.        ],
        [0.        , 0.        , 0.25575282]]))

In [17]:


import sympy as sp

def circle_intersectoins(circle_1_points, circle_2_points):
    """
    Calculate the intersection points of two circles.

    Parameters:
    circle_1_points (tuple): The center coordinates (x, y) and radius of the first circle.
    circle_2_points (tuple): The center coordinates (x, y) and radius of the second circle.
    
    Returns:
    list: A list of tuples representing the coordinates of the intersection points.
    """
    # circulate to get the point of intersection
    c1 = sp.Circle((circle_1_points[0], circle_1_points[1]), circle_1_points[2])
    c2 = sp.Circle((circle_2_points[0], circle_2_points[1]), circle_2_points[2])
    intersection_points = c1.intersection(c2)
    # get the coordinates of the intersection points
    intersection_points = [(point.x, point.y) for point in [point.evalf() for point in intersection_points]]
    return intersection_points

landmark_1_x =0
landmark_1_y = 0
landmark_1_range = 4
landmark_2_x = 5
landmark_2_y = 5
landmark_2_range = 4
circle_1_points = [landmark_1_x, landmark_1_y, landmark_1_range]
circle_2_points = [landmark_2_x, landmark_2_y, landmark_2_range]
intersection_points = circle_intersectoins(circle_1_points, circle_2_points)
intersection_points

[(1.17712434446770, 3.82287565553230), (3.82287565553230, 1.17712434446770)]

In [33]:
import sympy as sp
import numpy as np
def atan2(x, y):
    """Extends the inverse tangent function to all four quadrants."""
    # print(x, y)
    if x == 0:
        if y == 0:
            return 0
        else:
            return np.sign(y) * np.pi / 2
    elif x > 0:
        return np.arctan(float(y / x))
    else:
        return np.sign(y) * (np.pi - np.arctan(abs(float(y / x))))

def circle_intersectoins(circle_1_points, circle_2_points):
    """
    Calculate the intersection points of two circles.

    Parameters:
    circle_1_points (tuple): The center coordinates (x, y) and radius of the first circle.
    circle_2_points (tuple): The center coordinates (x, y) and radius of the second circle.
    
    Returns:
    list: A list of tuples representing the coordinates of the intersection points.
    """
    # circulate to get the point of intersection
    c1 = sp.Circle((circle_1_points[0], circle_1_points[1]), circle_1_points[2])
    c2 = sp.Circle((circle_2_points[0], circle_2_points[1]), circle_2_points[2])
    intersection_points = c1.intersection(c2)
    # get the coordinates of the intersection points
    intersection_points = [(round(point.x,2),round(point.y,2)) for point in [point.evalf() for point in intersection_points]]
    return intersection_points

In [34]:
detected_landmarks = [[300, 100, 0, 141.4213562373095, -0.7853981633974483], [300, 500, 1, 316.22776601683796, 1.2490457723982544], [500, 700, 2, 583.09518948453, 1.0303768265243125], [500, 300, 3, 316.22776601683796, 0.3217505543966422], [700, 100, 4, 509.9019513592785, -0.19739555984988078], [700, 500, 5, 583.09518948453, 0.5404195002705842], [100, 100, 6, 141.4213562373095, -2.356194490192345], [100, 700, 7, 509.9019513592785, 1.7681918866447772], [900, 100, 8, 707.1067811865476, -0.1418970546041639], [900, 700, 9, 860.2325267042627, 0.6202494859828215]]
# get the last 2 landmarks
detected_landmarks = detected_landmarks[:-2]
beacon_1 = detected_landmarks[0]
beacon_2 = detected_landmarks[1]
# get the two points of intersection of the two circles
pos_points = circle_intersectoins([beacon_1[0], beacon_1[1], beacon_1[3]], [beacon_2[0], beacon_2[1], beacon_2[3]])
pos_points

[(200.000000000000, 200.000000000000), (400.000000000000, 200.000000000000)]

In [40]:
delta_min = np.inf
for x,y in pos_points:
    print(x,y)
    print(f"beacon_1: {beacon_1[0], beacon_1[1], beacon_1[4]}")
    print(f"beacon_2: {beacon_2[0], beacon_2[1], beacon_2[4]}")
    print(atan2(beacon_1[0] - x, beacon_1[1]- y) -  beacon_1[4])
    print(atan2(beacon_2[0] - x, beacon_2[1] - y) - beacon_2[4])

    delta = abs((atan2(beacon_1[0] - x, beacon_1[1]- y) -  beacon_1[4]) 
                - (atan2(beacon_2[0] - x, beacon_2[1] - y) - beacon_2[4]))
    print(delta)
    if delta < delta_min:
        delta_min = delta
        bel_pos_x = x
        bel_pos_y = y
        
bel_theta = (atan2(beacon_1[0] - bel_pos_x, beacon_1[1]- bel_pos_y) -  beacon_1[4])
print("Final position: ", bel_pos_x, bel_pos_y, bel_theta)


200.000000000000 200.000000000000
beacon_1: (300, 100, -0.7853981633974483)
beacon_2: (300, 500, 1.2490457723982544)
0.0
0.0
0.0
400.000000000000 200.000000000000
beacon_1: (300, 100, -0.7853981633974483)
beacon_2: (300, 500, 1.2490457723982544)
-1.5707963267948966
0.6435011087932843
2.214297435588181
Final position:  200.000000000000 200.000000000000 0.0


In [18]:
x, y  = -100.000000000001, 100.000000000000
np.arctan(abs(y / x))

0.7853981633974433

In [2]:
import sympy as sp
def point_on_line(point, line):
    """
    Check if a point lies on a line defined by two points.

    Parameters:
    point (tuple): The coordinates of the point.
    line (tuple): The coordinates of the two points defining the line.
    
    Returns:
    bool: True if the point lies on the line, False otherwise.
    """
    # create the line using the two points
    line = sp.Line(line[0], line[1])
    # create the point using the coordinates
    point = sp.Point(point)
    # check if the point lies on the line
    return line.contains(point)
point = (1,1)
line = ((0,0), (2,2))
point_on_line(point, line)


True

In [7]:
detected_landmarks = [2,3]
if len(detected_landmarks) >= 2:
    # get the last 2 landmarks
    detected_landmarks = detected_landmarks[-2:]
detected_landmarks

[2, 3]

In [5]:
np.random.normal(0, 0.5, 100)

array([ 0.09612686, -0.09591155,  0.23509782, -0.06702353,  0.14493494,
        0.24684562,  0.8361739 , -0.27415108,  0.56625954,  1.01049958,
       -0.671433  ,  0.07745794, -0.20444376,  0.43520446, -0.36435918,
       -0.2708616 ,  0.60504673, -0.16019392, -0.03407604, -0.01612747,
       -0.57760533, -0.40490252, -0.63645878, -0.49910351, -0.46878162,
        0.47084053,  0.24760893, -0.69679751, -0.8508698 ,  0.33878837,
        0.18724086,  0.41090958, -0.16747195, -0.35732207, -0.57258313,
        0.22868546, -0.01092495,  0.33799394, -0.08612942,  0.09121397,
        0.08543347,  0.12618827, -0.4715169 ,  0.7811177 , -0.8394819 ,
        0.51790447,  0.4459283 ,  0.2489163 ,  0.06026367,  0.42450078,
        0.45162359, -0.7308123 , -0.23024212,  0.6928438 , -1.02530587,
        0.8981935 , -0.3730893 , -0.28235911,  1.00233968,  0.33737649,
       -0.12809009, -0.49126745, -0.14297473, -0.67784777, -0.8027795 ,
        0.37143629,  0.51561046, -0.05656401,  0.03208754, -0.28

In [50]:
# sympy intersection of two lines
import sympy as sp
import numpy as np

# wriet a function to get the intersection of two lines and handle all the corner cases
def line_line_intersections(line1, line2):
    # define env_line
    env_line = sp.Line(sp.Point(line1[0][0], line1[0][1]), sp.Point(line1[1][0], line1[1][1]))
    l2 = sp.Line(sp.Point(line2[0][0], line2[0][1]), sp.Point(line2[1][0], line2[1][1]))
    intersection = env_line.intersection(l2)
    # get the coordinates of the intersection point as a tuple of floats
    if len(intersection) == 1 and isinstance(intersection[0], sp.Point):
        return (float(intersection[0].x), float(intersection[0].y))
    return None

# time the function
import time
start_time = time.time()
# test
line1 = [(0, 0), (2, 2)]
line2 = [(0, 2), (2, 0)]
print(line_line_intersections(line1, line2))
print("--- %s seconds ---" % (time.time() - start_time))

(1.0, 1.0)
--- 0.002978801727294922 seconds ---


In [51]:
from shapely.geometry import LineString

def line_line_intersections_shap(line1, line2):

    line1 = LineString(line1)
    line2 = LineString(line2)

    intersection = line1.intersection(line2)

    if intersection.is_empty:
        return None
    elif intersection.geom_type == 'Point':
        return intersection.x, intersection.y
    elif intersection.geom_type == 'MultiPoint':
        # If there are multiple intersection points, return the first one
        return intersection[0].x, intersection[0].y
    else:
        return None
start_time = time.time()
# test
line1 = [(0, 0), (2, 2)]
# line2 = [(0, 2), (2, 0)]

print(line_line_intersections_shap(line1, line2))
print("--- %s seconds ---" % (time.time() - start_time))

(1.0, 1.0)
--- 0.0007700920104980469 seconds ---


In [52]:
a= [2,4,3]
b = [1,2,3]
a.extend(b)
a

[2, 4, 3, 1, 2, 3]